# Module 7: Task Automation Fundamentals

## Welcome to Module 7!

In Module 6, you mastered text processing. Now it's time to build **complete automation solutions** by combining everything you've learned.

### What You'll Learn

- Environment variables (Get/Set)
- Running external programs
- Capturing program output
- Error handling (Try/Catch/Finally)
- Creating functions
- Function parameters and validation
- Script parameters
- Building reusable scripts
- Creating a system maintenance script

### Why Task Automation Matters

Task automation lets you:
- **Combine** multiple operations into workflows
- **Handle** errors gracefully
- **Reuse** code with functions
- **Integrate** different tools and programs
- **Build** professional automation solutions

Let's build powerful automation!

## Setup: Prepare Practice Environment

In [1]:
import subprocess
from pathlib import Path

# Practice folder
practice_folder = Path.home() / "Documents" / "AutomationPractice"
practice_folder.mkdir(exist_ok=True)

# Create Automation practice subfolder
auto_practice = practice_folder / "Automation_Practice"
auto_practice.mkdir(exist_ok=True)

print(f"Practice folder ready: {auto_practice}")
print("Let's build automation solutions!\n")

# Helper function
def run_ps(command, cwd=None):
    """Run a PowerShell command and return output."""
    if cwd is None:
        cwd = str(auto_practice)
    
    result = subprocess.run(
        ['powershell', '-Command', command],
        cwd=cwd,
        capture_output=True,
        text=True,
        timeout=30
    )
    
    return result.stdout + result.stderr

print("✓ Helper function ready!")

Practice folder ready: C:\Users\USER\Documents\AutomationPractice\Automation_Practice
Let's build automation solutions!

✓ Helper function ready!


## 1. Environment Variables

Environment variables store system-wide settings accessible by all programs.

### Reading Environment Variables

In [2]:
output = run_ps('''
Write-Host "=== Common Environment Variables ==="
Write-Host ""

Write-Host "Username: $env:USERNAME"
Write-Host "Computer Name: $env:COMPUTERNAME"
Write-Host "User Profile: $env:USERPROFILE"
Write-Host "Temp Folder: $env:TEMP"
Write-Host "System Drive: $env:SystemDrive"
Write-Host "Number of Processors: $env:NUMBER_OF_PROCESSORS"

Write-Host ""
Write-Host "=== PATH Variable (first 5 entries) ==="
$pathEntries = $env:PATH -split ";"
$pathEntries | Select-Object -First 5 | ForEach-Object {
    Write-Host "  - $_"
}
Write-Host "  ... and $($pathEntries.Count - 5) more"
''')

print(output)

=== Common Environment Variables ===

Username: USER
Computer Name: LAPTOP-D9EUFCH4
User Profile: C:\Users\USER
Temp Folder: C:\Users\USER\AppData\Local\Temp
System Drive: C:
Number of Processors: 16

=== PATH Variable (first 5 entries) ===
  - C:\Users\USER\bin
  - C:\Program Files\Git\mingw64\bin
  - C:\Program Files\Git\usr\local\bin
  - C:\Program Files\Git\usr\bin
  - C:\Program Files\Git\usr\bin
  ... and 65 more



### Setting Environment Variables (Process Scope)

In [3]:
output = run_ps('''
Write-Host "=== Set Environment Variables ==="
Write-Host ""

# Set process-level environment variable
$env:MY_APP_CONFIG = "C:\\Config\\app.config"
$env:MY_APP_DEBUG = "true"
$env:MY_APP_VERSION = "1.0.0"

Write-Host "Set custom variables:"
Write-Host "  MY_APP_CONFIG = $env:MY_APP_CONFIG"
Write-Host "  MY_APP_DEBUG = $env:MY_APP_DEBUG"
Write-Host "  MY_APP_VERSION = $env:MY_APP_VERSION"

Write-Host ""
Write-Host "Note: These variables only exist in current PowerShell session"
''')

print(output)

=== Set Environment Variables ===

Set custom variables:
  MY_APP_CONFIG = C:\Config\app.config
  MY_APP_DEBUG = true
  MY_APP_VERSION = 1.0.0

Note: These variables only exist in current PowerShell session



## 2. Running External Programs

Execute external programs and capture their output.

### Basic Program Execution

In [4]:
output = run_ps('''
Write-Host "=== Run External Programs ==="
Write-Host ""

# Run ipconfig
Write-Host "Running ipconfig /all:"
Write-Host "====================="
$result = ipconfig /all
$result | Select-Object -First 10
Write-Host "... (truncated)"

Write-Host ""
Write-Host "Running systeminfo (basic info):"
Write-Host "================================"
$sysInfo = systeminfo | Select-Object -First 5
$sysInfo
''')

print(output)

=== Run External Programs ===

Running ipconfig /all:

Windows IP Configuration

   Host Name . . . . . . . . . . . . : LAPTOP-D9EUFCH4
   Primary Dns Suffix  . . . . . . . : 
   Node Type . . . . . . . . . . . . : Hybrid
   IP Routing Enabled. . . . . . . . : No
   WINS Proxy Enabled. . . . . . . . : No
   DNS Suffix Search List. . . . . . : realtek

... (truncated)

Running systeminfo (basic info):

Host Name:                     LAPTOP-D9EUFCH4
OS Name:                       Microsoft Windows 11 Home Single Language
OS Version:                    10.0.26200 N/A Build 26200
OS Manufacturer:               Microsoft Corporation



### Capturing Program Output

In [5]:
output = run_ps('''
Write-Host "=== Capture and Process Output ==="
Write-Host ""

# Capture directory listing
$output = cmd /c dir

Write-Host "Total lines of output: $($output.Count)"
Write-Host ""

# Process output
Write-Host "First 5 lines:"
$output | Select-Object -First 5

Write-Host ""
Write-Host "Lines containing 'File':"
$output | Where-Object {$_ -like "*File*"}
''')

print(output)

=== Capture and Process Output ===

Total lines of output: 9

First 5 lines:
 Volume in drive C is OS
 Volume Serial Number is 6AEB-E14D

 Directory of C:\Users\USER\Documents\AutomationPractice\Automation_Practice


Lines containing 'File':
               0 File(s)              0 bytes



### Using Start-Process

In [6]:
output = run_ps('''
Write-Host "=== Start-Process Examples ==="
Write-Host ""

# Run notepad and wait for it to close (will open and close immediately in automation)
Write-Host "Starting notepad (will close automatically)..."
Start-Process notepad -Wait -WindowStyle Hidden
Write-Host "Notepad closed"

Write-Host ""
Write-Host "Running cmd command with Start-Process:"
Start-Process cmd -ArgumentList "/c", "echo Hello from CMD" -Wait -NoNewWindow

Write-Host "Command completed"
''')

print(output)

=== Start-Process Examples ===

Starting notepad (will close automatically)...
Notepad closed

Running cmd command with Start-Process:
Hello from CMD 
Command completed
Start-Process : This command cannot be run completely because the system cannot find all the information required.
At line:7 char:1
+ Start-Process notepad -Wait -WindowStyle Hidden
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 



## 3. Error Handling with Try/Catch

Handle errors gracefully to make scripts robust.

### Basic Try/Catch

In [7]:
output = run_ps('''
Write-Host "=== Try/Catch Error Handling ==="
Write-Host ""

# Example 1: Successful operation
Write-Host "Attempting to read existing file:"
try {
    # Create a test file first
    "Test content" | Out-File "test.txt"
    
    $content = Get-Content "test.txt" -ErrorAction Stop
    Write-Host "Success! Content: $content"
}
catch {
    Write-Host "Error occurred: $($_.Exception.Message)" -ForegroundColor Red
}

Write-Host ""

# Example 2: Failed operation
Write-Host "Attempting to read non-existent file:"
try {
    $content = Get-Content "nonexistent.txt" -ErrorAction Stop
    Write-Host "Success! Content: $content"
}
catch {
    Write-Host "Error caught: $($_.Exception.Message)" -ForegroundColor Yellow
    Write-Host "Continuing script execution..." -ForegroundColor Green
}
''')

print(output)

=== Try/Catch Error Handling ===

Attempting to read existing file:
Success! Content: Test content

Attempting to read non-existent file:
Error caught: Cannot find path 'C:\Users\USER\Documents\AutomationPractice\Automation_Practice\nonexistent.txt' because it does not exist.
Continuing script execution...



### Try/Catch/Finally

In [8]:
output = run_ps('''
Write-Host "=== Try/Catch/Finally ==="
Write-Host ""

$logFile = "operation.log"

try {
    Write-Host "Starting operation..."
    "[START] Operation started" | Out-File $logFile
    
    # Simulate some work
    Write-Host "Processing..."
    "[INFO] Processing data" | Add-Content $logFile
    
    # Simulate an error
    Write-Host "Attempting risky operation..."
    throw "Simulated error for demonstration"
    
    Write-Host "This line won't execute"
}
catch {
    Write-Host "Error caught: $($_.Exception.Message)" -ForegroundColor Red
    "[ERROR] $($_.Exception.Message)" | Add-Content $logFile
}
finally {
    Write-Host "Cleanup: Closing log file" -ForegroundColor Cyan
    "[END] Operation completed" | Add-Content $logFile
    
    Write-Host ""
    Write-Host "Log file contents:"
    Get-Content $logFile
}
''')

print(output)

=== Try/Catch/Finally ===

Starting operation...
Processing...
Attempting risky operation...
Error caught: Simulated error for demonstration
Cleanup: Closing log file

Log file contents:
[START] Operation started
[INFO] Processing data
[ERROR] Simulated error for demonstration
[END] Operation completed



### Specific Error Types

In [9]:
output = run_ps('''
Write-Host "=== Catching Specific Errors ==="
Write-Host ""

try {
    # Try to divide by zero
    $result = 10 / 0
}
catch [System.DivideByZeroException] {
    Write-Host "Caught divide by zero error!" -ForegroundColor Yellow
    Write-Host "Using default value instead"
    $result = 0
}
catch {
    Write-Host "Caught other error: $($_.Exception.Message)" -ForegroundColor Red
}

Write-Host "Result: $result"
''')

print(output)

=== Catching Specific Errors ===

Caught divide by zero error!
Using default value instead
Result: 0



## 4. Creating Functions

Functions make code reusable and organized.

### Basic Functions

In [10]:
output = run_ps('''
Write-Host "=== Basic Functions ==="
Write-Host ""

# Simple function
function Say-Hello {
    Write-Host "Hello from PowerShell!"
}

# Function with parameters
function Greet-Person {
    param($Name)
    Write-Host "Hello, $Name!"
}

# Function with return value
function Get-Square {
    param($Number)
    return $Number * $Number
}

# Call functions
Say-Hello
Greet-Person -Name "Alice"
Greet-Person -Name "Bob"

$squared = Get-Square -Number 7
Write-Host "7 squared = $squared"
''')

print(output)

=== Basic Functions ===

Hello from PowerShell!
Hello, Alice!
Hello, Bob!
7 squared = 49



### Functions with Multiple Parameters

In [11]:
output = run_ps('''
Write-Host "=== Functions with Multiple Parameters ==="
Write-Host ""

function Calculate-Rectangle {
    param(
        $Length,
        $Width
    )
    
    $area = $Length * $Width
    $perimeter = 2 * ($Length + $Width)
    
    Write-Host "Rectangle: ${Length} x ${Width}"
    Write-Host "  Area: $area"
    Write-Host "  Perimeter: $perimeter"
}

# Call with different values
Calculate-Rectangle -Length 10 -Width 5
Write-Host ""
Calculate-Rectangle -Length 8 -Width 8
''')

print(output)

=== Functions with Multiple Parameters ===

Rectangle: 10 x 5
  Area: 50
  Perimeter: 30

Rectangle: 8 x 8
  Area: 64
  Perimeter: 32



### Functions with Default Values

In [12]:
output = run_ps('''
Write-Host "=== Functions with Default Values ==="
Write-Host ""

function Write-Log {
    param(
        $Message,
        $Level = "INFO",
        $ShowTimestamp = $true
    )
    
    $timestamp = if ($ShowTimestamp) {
        (Get-Date -Format "yyyy-MM-dd HH:mm:ss")
    } else {
        ""
    }
    
    $logLine = "$timestamp [$Level] $Message".Trim()
    Write-Host $logLine
}

# Call with different parameters
Write-Log -Message "Application started"
Write-Log -Message "Warning: Low disk space" -Level "WARNING"
Write-Log -Message "Error occurred" -Level "ERROR"
Write-Log -Message "Simple message" -ShowTimestamp $false
''')

print(output)

=== Functions with Default Values ===

2025-11-14 23:07:18 [INFO] Application started
2025-11-14 23:07:18 [WARNING] Warning: Low disk space
2025-11-14 23:07:18 [ERROR] Error occurred
[INFO] Simple message



### Functions with Parameter Validation

In [13]:
output = run_ps('''
Write-Host "=== Parameter Validation ==="
Write-Host ""

function Get-FileSize {
    param(
        [Parameter(Mandatory=$true)]
        [string]$Path,
        
        [ValidateSet("Bytes", "KB", "MB", "GB")]
        [string]$Unit = "Bytes"
    )
    
    if (-not (Test-Path $Path)) {
        Write-Host "Error: File not found: $Path" -ForegroundColor Red
        return
    }
    
    $file = Get-Item $Path
    $size = $file.Length
    
    $displaySize = switch ($Unit) {
        "KB" { $size / 1KB }
        "MB" { $size / 1MB }
        "GB" { $size / 1GB }
        default { $size }
    }
    
    Write-Host "$($file.Name): $([math]::Round($displaySize, 2)) $Unit"
}

# Create test files
"Small file" | Out-File "small.txt"
"Large file" * 1000 | Out-File "large.txt"

# Call function
Get-FileSize -Path "small.txt"
Get-FileSize -Path "small.txt" -Unit "KB"
Get-FileSize -Path "large.txt" -Unit "KB"
''')

print(output)

=== Parameter Validation ===

small.txt: 26 Bytes
small.txt: 0.03 KB
large.txt: 19.54 KB



## 5. Advanced Function Example: Backup Files

In [14]:
output = run_ps('''
Write-Host "=== Backup Function ==="
Write-Host ""

function Backup-Files {
    param(
        [Parameter(Mandatory=$true)]
        [string]$SourcePath,
        
        [string]$BackupPath = "Backup",
        
        [string]$FilePattern = "*.*",
        
        [switch]$IncludeTimestamp
    )
    
    try {
        # Validate source
        if (-not (Test-Path $SourcePath)) {
            throw "Source path not found: $SourcePath"
        }
        
        # Create backup folder
        if ($IncludeTimestamp) {
            $timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
            $BackupPath = "${BackupPath}_${timestamp}"
        }
        
        if (-not (Test-Path $BackupPath)) {
            New-Item -Path $BackupPath -ItemType Directory | Out-Null
        }
        
        # Get files
        $files = Get-ChildItem -Path $SourcePath -Filter $FilePattern -File
        
        Write-Host "Backing up $($files.Count) files from $SourcePath"
        Write-Host "Destination: $BackupPath"
        Write-Host ""
        
        # Copy files
        $copied = 0
        foreach ($file in $files) {
            Copy-Item -Path $file.FullName -Destination $BackupPath -Force
            $copied++
            Write-Host "  Copied: $($file.Name)"
        }
        
        Write-Host ""
        Write-Host "Backup complete: $copied files copied" -ForegroundColor Green
        
        return @{
            Success = $true
            FilesCopied = $copied
            BackupPath = $BackupPath
        }
    }
    catch {
        Write-Host "Backup failed: $($_.Exception.Message)" -ForegroundColor Red
        return @{
            Success = $false
            Error = $_.Exception.Message
        }
    }
}

# Create test files
"Source Folder" | Out-Null
New-Item -Path "Source" -ItemType Directory -Force | Out-Null
1..5 | ForEach-Object {
    $fileName = "file$_.txt"
    $filePath = Join-Path "Source" $fileName
    "File $_" | Out-File $filePath
}

# Test the function
$result = Backup-Files -SourcePath "Source" -IncludeTimestamp

Write-Host ""
Write-Host "Result:"
Write-Host "  Success: $($result.Success)"
Write-Host "  Files Copied: $($result.FilesCopied)"
Write-Host "  Backup Location: $($result.BackupPath)"
''')

print(output)

=== Backup Function ===

Backing up 5 files from Source
Destination: Backup_20251114_230722

  Copied: file1.txt
  Copied: file2.txt
  Copied: file3.txt
  Copied: file4.txt
  Copied: file5.txt

Backup complete: 5 files copied

Result:
  Success: True
  Files Copied: 5
  Backup Location: Backup_20251114_230722



## 6. Script Parameters

Make scripts accept parameters like functions.

In [15]:
# Create a parameterized script
script_content = '''param(
    [Parameter(Mandatory=$true)]
    [string]$Name,
    
    [int]$Count = 3,
    
    [switch]$Verbose
)

if ($Verbose) {
    Write-Host "Script started with parameters:"
    Write-Host "  Name: $Name"
    Write-Host "  Count: $Count"
    Write-Host ""
}

for ($i = 1; $i -le $Count; $i++) {
    Write-Host "${i}. Hello, $Name!"
}

if ($Verbose) {
    Write-Host ""
    Write-Host "Script completed"
}'''

(auto_practice / "greet.ps1").write_text(script_content, encoding='utf-8')

output = run_ps('''
Write-Host "=== Script Parameters ==="
Write-Host ""

Write-Host "Running script with different parameters:"
Write-Host "=========================================="
Write-Host ""

Write-Host "1. Basic call:"
.\greet.ps1 -Name "Alice"

Write-Host ""
Write-Host "2. With custom count:"
.\greet.ps1 -Name "Bob" -Count 5

Write-Host ""
Write-Host "3. With verbose output:"
.\greet.ps1 -Name "Charlie" -Count 2 -Verbose
''')

print(output)

<>:38: SyntaxWarning: invalid escape sequence '\g'
<>:38: SyntaxWarning: invalid escape sequence '\g'
C:\Users\USER\AppData\Local\Temp\ipykernel_416\3277831922.py:38: SyntaxWarning: invalid escape sequence '\g'
  .\greet.ps1 -Name "Alice"


=== Script Parameters ===

Running script with different parameters:

1. Basic call:

2. With custom count:

3. With verbose output:
.\greet.ps1 : File C:\Users\USER\Documents\AutomationPractice\Automation_Practice\greet.ps1 cannot be loaded because 
running scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:10 char:1
+ .\greet.ps1 -Name "Alice"
+ ~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess
.\greet.ps1 : File C:\Users\USER\Documents\AutomationPractice\Automation_Practice\greet.ps1 cannot be loaded because 
running scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:14 char:1
+ .\greet.ps1 -Name "Bob" -Count 5
+ ~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualif

## 7. Practical Example: System Maintenance Script

Build a comprehensive maintenance script combining everything learned.

In [16]:
# Create comprehensive maintenance script
maintenance_script = '''# System Maintenance Script
param(
    [switch]$CleanTemp,
    [switch]$CheckDisk,
    [switch]$BackupLogs,
    [string]$ReportPath = "maintenance_report.txt"
)

$report = @()
$report += "="*60
$report += "SYSTEM MAINTENANCE REPORT"
$report += "Generated: $(Get-Date -Format 'yyyy-MM-dd HH:mm:ss')"
$report += "="*60
$report += ""

function Write-ReportLine {
    param($Message)
    $script:report += $Message
    Write-Host $Message
}

# Task 1: Clean Temp Files
if ($CleanTemp) {
    Write-ReportLine "TASK: Clean Temporary Files"
    Write-ReportLine "-"*60
    
    try {
        # Create temp files for demo
        1..5 | ForEach-Object {
            "Temp $_" | Out-File "temp$_.tmp"
        }
        
        $tempFiles = Get-ChildItem -Filter "*.tmp"
        $tempSize = ($tempFiles | Measure-Object -Property Length -Sum).Sum
        
        Write-ReportLine "Found $($tempFiles.Count) temp files ($tempSize bytes)"
        
        $tempFiles | Remove-Item -Force
        Write-ReportLine "Deleted $($tempFiles.Count) temporary files"
        Write-ReportLine "Status: SUCCESS"
    }
    catch {
        Write-ReportLine "Error: $($_.Exception.Message)"
        Write-ReportLine "Status: FAILED"
    }
    Write-ReportLine ""
}

# Task 2: Check Disk Space
if ($CheckDisk) {
    Write-ReportLine "TASK: Check Disk Space"
    Write-ReportLine "-"*60
    
    try {
        $drive = Get-PSDrive -PSProvider FileSystem | Where-Object {$_.Name -eq "C"}
        $freeGB = [math]::Round($drive.Free / 1GB, 2)
        $usedGB = [math]::Round($drive.Used / 1GB, 2)
        $totalGB = [math]::Round(($drive.Free + $drive.Used) / 1GB, 2)
        $percentFree = [math]::Round(($drive.Free / ($drive.Free + $drive.Used)) * 100, 1)
        
        Write-ReportLine "Drive C:"
        Write-ReportLine "  Total: $totalGB GB"
        Write-ReportLine "  Used: $usedGB GB"
        Write-ReportLine "  Free: $freeGB GB ($percentFree%)"
        
        if ($percentFree -lt 10) {
            Write-ReportLine "  WARNING: Low disk space!" 
        } else {
            Write-ReportLine "  Disk space OK"
        }
        Write-ReportLine "Status: SUCCESS"
    }
    catch {
        Write-ReportLine "Error: $($_.Exception.Message)"
        Write-ReportLine "Status: FAILED"
    }
    Write-ReportLine ""
}

# Task 3: Backup Logs
if ($BackupLogs) {
    Write-ReportLine "TASK: Backup Log Files"
    Write-ReportLine "-"*60
    
    try {
        # Create sample log files
        "Log entry 1" | Out-File "app.log"
        "Log entry 2" | Out-File "error.log"
        
        $logFiles = Get-ChildItem -Filter "*.log"
        
        if ($logFiles.Count -eq 0) {
            Write-ReportLine "No log files found"
        } else {
            $backupFolder = "LogBackup_$(Get-Date -Format 'yyyyMMdd')"
            New-Item -Path $backupFolder -ItemType Directory -Force | Out-Null
            
            foreach ($log in $logFiles) {
                Copy-Item -Path $log.FullName -Destination $backupFolder
            }
            
            Write-ReportLine "Backed up $($logFiles.Count) log files to $backupFolder"
        }
        Write-ReportLine "Status: SUCCESS"
    }
    catch {
        Write-ReportLine "Error: $($_.Exception.Message)"
        Write-ReportLine "Status: FAILED"
    }
    Write-ReportLine ""
}

# Summary
$report += "="*60
$report += "MAINTENANCE COMPLETED"
$report += "Report saved to: $ReportPath"
$report += "="*60

# Save report
$report | Out-File $ReportPath -Encoding UTF8

Write-Host ""
Write-Host "Report saved to: $ReportPath" -ForegroundColor Green
'''

(auto_practice / "maintenance.ps1").write_text(maintenance_script, encoding='utf-8')

output = run_ps('''
Write-Host "=== System Maintenance Script ==="
Write-Host ""

.\maintenance.ps1 -CleanTemp -CheckDisk -BackupLogs
''')

print(output)

<>:133: SyntaxWarning: invalid escape sequence '\m'
<>:133: SyntaxWarning: invalid escape sequence '\m'
C:\Users\USER\AppData\Local\Temp\ipykernel_416\1266693491.py:133: SyntaxWarning: invalid escape sequence '\m'
  .\maintenance.ps1 -CleanTemp -CheckDisk -BackupLogs


=== System Maintenance Script ===

.\maintenance.ps1 : File C:\Users\USER\Documents\AutomationPractice\Automation_Practice\maintenance.ps1 cannot be 
loaded because running scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:5 char:1
+ .\maintenance.ps1 -CleanTemp -CheckDisk -BackupLogs
+ ~~~~~~~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess



## Summary and Key Takeaways

Congratulations! You've mastered task automation fundamentals.

### What You Learned:

✓ **Environment Variables** - Reading and setting  
✓ **External Programs** - Running and capturing output  
✓ **Error Handling** - Try/Catch/Finally blocks  
✓ **Functions** - Creating reusable code  
✓ **Parameters** - Function and script parameters  
✓ **Validation** - Parameter validation  
✓ **Real-World Projects** - Complete maintenance script  

### Quick Reference: Automation

```powershell
# Environment Variables
$env:USERNAME
$env:TEMP
$env:MY_VAR = "value"

# Run Programs
$output = ipconfig /all
Start-Process notepad -Wait

# Error Handling
try {
    # risky code
}
catch {
    Write-Host "Error: $($_.Exception.Message)"
}
finally {
    # cleanup
}

# Functions
function Do-Something {
    param(
        [Parameter(Mandatory=$true)]
        [string]$Name,
        
        [int]$Count = 1
    )
    
    # function code
}

# Script Parameters
param(
    [Parameter(Mandatory=$true)]
    [string]$Path,
    
    [switch]$Verbose
)
```

### Best Practices

1. **Always use error handling** for production scripts
2. **Validate parameters** to catch issues early
3. **Use functions** to organize code
4. **Add verbose output** for debugging
5. **Log operations** to track what happened
6. **Test with different inputs** before deploying

### Next Steps

In **Module 8: Scheduled Tasks**, you'll learn:
- Creating scheduled tasks
- Task triggers (time, event-based)
- Running scripts automatically
- Building automated workflows

You're now ready to build professional automation solutions!

## Cleanup

Run this cell to remove all practice files:

In [17]:
import shutil

print("Cleaning up Automation_Practice folder...\n")

if auto_practice.exists():
    shutil.rmtree(auto_practice)
    print(f"✓ Removed {auto_practice}")
    print("\nAll practice files deleted.")
    print("The main AutomationPractice folder remains for future modules.")
else:
    print("Practice folder already cleaned up!")

Cleaning up Automation_Practice folder...

✓ Removed C:\Users\USER\Documents\AutomationPractice\Automation_Practice

All practice files deleted.
The main AutomationPractice folder remains for future modules.
